# Accelerating Python with Cython

## Baseline (step 0)

In [ ]:
import numpy as np

def f(x):
    return x ** 2 - x

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

def apply_integrate_f(col_a, col_b, col_N):
    n = len(col_N)
    res = np.empty(n,dtype=np.float64)
    for i in range(n):
        res[i] = integrate_f(col_a[i], col_b[i], col_N[i])
    return res

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "a": np.random.randn(1000),
        "b": np.random.randn(1000),
        "N": np.random.randint(low=100, high=1000, size=1000)
    }
)

In [ ]:
%timeit apply_integrate_f(df['a'], df['b'], df['N'])

## Cython: Benchmarking (step 1)

In [ ]:
%load_ext cython

Run
```
%load ../content/example/cython/integrate_cython_step1.py
```

In [ ]:
%load ../content/example/cython/integrate_cython_step1.py

In [ ]:
%timeit apply_integrate_f_cython_step1(df['a'], df['b'], df['N'])

## Cython: Adding data type annotation to input variables (step 2)

Run either
```
%load ../content/example/cython/integrate_cython_step2.py
%load ../content/example/cython/integrate_cython_step2_purepy.py
```

In [ ]:
%load ../content/example/cython/integrate_cython_step2_purepy.py

In [ ]:
%timeit apply_integrate_f_cython_step2(df['a'].to_numpy(), df['b'].to_numpy(), df['N'].to_numpy())

## Cython: Adding data type annotation to functions (step 3)

Run either
```
%load ../content/example/cython/integrate_cython_step3.py
%load ../content/example/cython/integrate_cython_step3_purepy.py
```

In [ ]:
%load ../content/example/cython/integrate_cython_step3_purepy.py

In [ ]:
%timeit apply_integrate_f_cython_step3(df['a'].to_numpy(), df['b'].to_numpy(), df['N'].to_numpy())

## Cython: Adding data type annotation to local variables (step 4)

Run either
```
%load ../content/example/cython/integrate_cython_step4.py
%load ../content/example/cython/integrate_cython_step4_purepy.py
```

In [ ]:
%load ../content/example/cython/integrate_cython_step4_purepy.py

In [ ]:
%timeit apply_integrate_f_cython_step4(df['a'].to_numpy(), df['b'].to_numpy(), df['N'].to_numpy())

## Demo: Other useful features

In [ ]:
%%cython

import cython
from cython.parallel import parallel, prange
from cython.cimports.libc.math import sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def normalize(x: cython.double[:]):
   """Normalize a 1D array by dividing all its elements using its root-mean-square (RMS) value."""
   i: cython.Py_ssize_t
   total: cython.double = 0
   norm: cython.double
   with cython.nogil, parallel():
      for i in prange(x.shape[0]):
            total += x[i]*x[i]
      norm = sqrt(total)
      for i in prange(x.shape[0]):
            x[i] /= norm

In [ ]:
import numpy as np

def normalize_numpy(x):
    total = np.dot(x, x)
    norm = total ** 0.5

    x[:] /= norm

In [ ]:
from math import sqrt

def normalize_naive(x):
    total = 0
    for i in range(x.shape[0]):
        total += x[i] * x[i]

    norm = sqrt(total)
    for i in range(x.shape[0]):
        x[i] /= norm

In [ ]:
a = 10
b = 100
xs = (b - a) * np.random.random_sample(size=100_000) + a
xs.mean()

In [ ]:
xs_copy = xs.copy()
%time normalize(xs_copy)
xs_copy.mean()

In [ ]:
xs_copy = xs.copy()
%time normalize_numpy(xs_copy)
xs_copy.mean()

In [ ]:
xs_copy = xs.copy()
%time normalize_naive(xs_copy)
xs_copy.mean()